## ANALYSE PIXELS

In [1]:
#### ANALYSE DES PIXELS FRAME PAR FRAME
import cv2
import numpy as np

video_path = 'penalty5.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Erreur: Impossible de lire la vidéo.")
    exit()

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

print(f"Dimensions de la vidéo : {width}x{height}, FPS : {fps}")

output_path = 'penalty5_pixel.mp4'  
fourcc = cv2.VideoWriter_fourcc(*'H264')  # Utiliser XVID comme codec
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

frame_count = 0
prev_frame = None

while True:
    # Lire la frame actuelle
    ret, frame = cap.read()

    # Si la vidéo est terminée, quitter
    if not ret:
        print("Fin de la vidéo ou erreur de lecture.")
        break

    # Convertir en niveaux de gris pour simplifier la détection de variations
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if prev_frame is not None:
        # Calculer la différence absolue entre la frame actuelle et la précédente
        diff = cv2.absdiff(prev_frame, gray_frame)

        # Appliquer un seuil pour créer une image binaire (pour détecter des changements significatifs)
        _, thresh = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)

        # Trouver les contours des zones où il y a des différences
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Dessiner les contours sur la frame originale (en couleur)
        annotated_frame = frame.copy()
        cv2.drawContours(annotated_frame, contours, -1, (0, 255, 0), 2)

        # Si une variation importante est détectée, ajouter une annotation texte
        if np.sum(diff) > 5000:  # seuil arbitraire
            cv2.putText(annotated_frame, f"Variation detected at frame {frame_count}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        # Enregistrer la frame annotée dans le fichier vidéo
        out.write(annotated_frame)

    # Mettre à jour la frame précédente
    prev_frame = gray_frame
    frame_count += 1

# Libérer les ressources
cap.release()
out.release()
cv2.destroyAllWindows()

print("Video processed and saved successfully.")


Dimensions de la vidéo : 1920x1080, FPS : 30
Fin de la vidéo ou erreur de lecture.
Video processed and saved successfully.


## BOX DE COORDONNEES

In [1]:
#### ANALYSE DES PIXELS FRAME PAR FRAME
import cv2
import numpy as np

video_path = 'penalty1.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Erreur: Impossible de lire la vidéo.")
    exit()

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

print(f"Dimensions de la vidéo : {width}x{height}, FPS : {fps}")

output_path = 'penalty1_box.mp4'  
fourcc = cv2.VideoWriter_fourcc(*'H264')  
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

frame_count = 0
prev_frame = None

while True:
    ret, frame = cap.read()
    if not ret:
        print("Fin de la vidéo ou erreur de lecture.")
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if prev_frame is not None:
        diff = cv2.absdiff(prev_frame, gray_frame)
        _, thresh = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)

        # Trouver les contours des zones où il y a des différences
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        annotated_frame = frame.copy()

        # Parcourir les contours et créer des boîtes englobantes
        for contour in contours:
            # Obtenir les coordonnées de la boîte englobante
            x, y, w, h = cv2.boundingRect(contour)
            
            # Dessiner la boîte englobante sur la frame
            cv2.rectangle(annotated_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            
            # Ajouter un texte avec les coordonnées de la boîte
            cv2.putText(annotated_frame, f"x:{x}, y:{y}", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        if np.sum(diff) > 5000:
            cv2.putText(annotated_frame, f"Variation detected at frame {frame_count}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        out.write(annotated_frame)

    prev_frame = gray_frame
    frame_count += 1

cap.release()
out.release()
cv2.destroyAllWindows()

print("Video processed and saved successfully.")


Dimensions de la vidéo : 1920x1080, FPS : 30
Fin de la vidéo ou erreur de lecture.
Video processed and saved successfully.


## FILTERS : highest box

In [1]:
import cv2
import numpy as np

video_path = 'penalty5.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Erreur: Impossible de lire la vidéo.")
    exit()

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

print(f"Dimensions de la vidéo : {width}x{height}, FPS : {fps}")

output_path = 'test5.mp4'
fourcc = cv2.VideoWriter_fourcc(*'H264')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

frame_count = 0
prev_frame = None

while True:
    ret, frame = cap.read()
    if not ret:
        print("Fin de la vidéo ou erreur de lecture.")
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if prev_frame is not None:
        diff = cv2.absdiff(prev_frame, gray_frame)
        _, thresh = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)

        # Trouver les contours des zones où il y a des différences
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        annotated_frame = frame.copy()
        highest_box = None
        min_y = float('inf')

        # Parcourir les contours et déterminer la boîte avec le plus petit 'y'
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            if y < min_y:
                min_y = y
                highest_box = (x, y, w, h)

        # Si une boîte a été trouvée, dessiner la boîte la plus haute
        if highest_box is not None:
            x, y, w, h = highest_box
            cv2.rectangle(annotated_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(annotated_frame, f"x:{x}, y:{y}", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
            
            print(f"Coordonnées de la boîte la plus haute : x={x}, y={y}, w={w}, h={h}")

        out.write(annotated_frame)

    prev_frame = gray_frame
    frame_count += 1

cap.release()
out.release()
cv2.destroyAllWindows()

print("Vidéo annotée avec la boîte la plus haute sauvegardée avec succès.")


Dimensions de la vidéo : 1920x1080, FPS : 30
Coordonnées de la boîte la plus haute : x=1764, y=387, w=1, h=2
Coordonnées de la boîte la plus haute : x=621, y=358, w=1, h=2
Coordonnées de la boîte la plus haute : x=1207, y=398, w=1, h=1
Coordonnées de la boîte la plus haute : x=621, y=360, w=1, h=1
Coordonnées de la boîte la plus haute : x=24, y=214, w=3, h=3
Coordonnées de la boîte la plus haute : x=620, y=359, w=1, h=1
Coordonnées de la boîte la plus haute : x=1035, y=320, w=1, h=3
Coordonnées de la boîte la plus haute : x=670, y=236, w=2, h=2
Coordonnées de la boîte la plus haute : x=571, y=444, w=6, h=6
Coordonnées de la boîte la plus haute : x=1767, y=354, w=1, h=5
Coordonnées de la boîte la plus haute : x=921, y=356, w=1, h=1
Coordonnées de la boîte la plus haute : x=596, y=440, w=6, h=5
Coordonnées de la boîte la plus haute : x=1034, y=318, w=1, h=2
Coordonnées de la boîte la plus haute : x=1764, y=389, w=1, h=1
Coordonnées de la boîte la plus haute : x=1208, y=300, w=1, h=6
Coor

### Detection de plusieurs pénalités

#### Position initale en cliquant 

In [1]:
import cv2

# Variables globales
penalty_positions = []  # Liste pour stocker les positions et frames des pénalités

def click_event(event, x, y, flags, param):
    """
    Fonction de gestion des clics de souris.
    """
    global penalty_positions, frame_index, frame
    if event == cv2.EVENT_LBUTTONDOWN:  # Clic gauche
        penalty_positions.append((frame_index, (x, y)))
        print(f"Pénalité détectée à la frame {frame_index}, position : x={x}, y={y}")
        # Dessiner un point rouge sur la frame actuelle
        cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)  # Point rouge

# Charger la vidéo
video_path = 'penalty5.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Erreur: Impossible de lire la vidéo.")
    exit()

# Récupérer les dimensions et le FPS de la vidéo
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
print(f"Dimensions de la vidéo : {width}x{height}, FPS : {fps}")

frame_index = 0  # Indice de la frame actuelle

# Afficher la vidéo et attendre les clics
cv2.namedWindow("Video", cv2.WINDOW_NORMAL)  # Créer une fenêtre redimensionnable
cv2.resizeWindow("Video", width, height)  # Ajuster la taille de la fenêtre à celle de la vidéo
cv2.setMouseCallback("Video", click_event)  # Associer la gestion des clics

while True:
    ret, frame = cap.read()
    if not ret:
        print("Fin de la vidéo.")
        break

    # Afficher la vidéo avec les annotations (points rouges déjà ajoutés)
    cv2.imshow("Video", frame)
    frame_index += 1

    # Quitter si 'q' est pressé
    if cv2.waitKey(int(1000 / fps)) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Résultats finaux
print("Analyse terminée.")
print(f"Positions des pénalités détectées : {penalty_positions}")

# Sauvegarder les données dans un fichier texte
output_path = 'penalty_positions.txt'
with open(output_path, 'w') as file:
    for frame, position in penalty_positions:
        file.write(f"Frame: {frame}, Position: {position}\n")
print(f"Positions sauvegardées dans {output_path}")


Dimensions de la vidéo : 1920x1080, FPS : 30
Pénalité détectée à la frame 138, position : x=411, y=589
Pénalité détectée à la frame 249, position : x=382, y=597
Pénalité détectée à la frame 386, position : x=445, y=593
Pénalité détectée à la frame 566, position : x=398, y=588
Fin de la vidéo.
Analyse terminée.
Positions des pénalités détectées : [(138, (411, 589)), (249, (382, 597)), (386, (445, 593)), (566, (398, 588))]
Positions sauvegardées dans penalty_positions.txt


#### Decoupe des segements selon les cliques

In [3]:
import cv2
import os

# Créer les dossiers pour stocker les vidéos
os.makedirs("segments", exist_ok=True)

# Charger les positions des clics depuis le fichier texte
input_positions_file = 'penalty_positions.txt'
click_positions = []

with open(input_positions_file, 'r') as file:
    for line in file:
        parts = line.strip().split(',')
        frame = int(parts[0].split(':')[1].strip())
        click_positions.append(frame)

# Charger la vidéo originale
video_path = 'penalty5.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Erreur: Impossible de lire la vidéo.")
    exit()

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Calculer le nombre total de frames
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Vidéo : {width}x{height}, FPS : {fps}, Total de frames : {total_frames}")

# Ajouter une frame finale pour découper jusqu'à la fin de la vidéo
click_positions.append(total_frames)  # Dernier segment jusqu'à la fin de la vidéo

# Découper la vidéo en segments
segment_paths = []
for i in range(len(click_positions) - 1):
    start_frame = click_positions[i]
    end_frame = click_positions[i + 1]

    # Vérification des indices de frame
    if start_frame >= total_frames:
        print(f"Erreur : Le début du segment {i + 1} dépasse la fin de la vidéo (frame {start_frame}).")
        continue  # Sauter ce segment

    if end_frame > total_frames:
        end_frame = total_frames  # Ajuster la fin du segment à la dernière frame

    segment_path = f'segments/segment_{i + 1}.mp4'  # Utiliser .mp4 comme format de sortie
    segment_paths.append(segment_path)

    # Définir le writer vidéo pour le segment
    fourcc = cv2.VideoWriter_fourcc(*'H264')  # Utiliser H264 comme codec pour MP4
    out = cv2.VideoWriter(segment_path, fourcc, fps, (width, height))

    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    print(f"Découpage du segment {i + 1} : Frames {start_frame} à {end_frame}")

    for frame_num in range(start_frame, end_frame):
        ret, frame = cap.read()
        if not ret:
            print(f"Frame manquante à {frame_num}, segment {i + 1}.")
            break
        out.write(frame)

    out.release()
    print(f"Segment {i + 1} enregistré : {segment_path}")

print("Découpage terminé. Les segments sont stockés dans le dossier 'segments/'.")


Vidéo : 1920x1080, FPS : 30, Total de frames : 656
Découpage du segment 1 : Frames 37 à 123
Segment 1 enregistré : segments/segment_1.mp4
Découpage du segment 2 : Frames 123 à 247
Segment 2 enregistré : segments/segment_2.mp4
Découpage du segment 3 : Frames 247 à 382
Segment 3 enregistré : segments/segment_3.mp4
Découpage du segment 4 : Frames 382 à 561
Segment 4 enregistré : segments/segment_4.mp4
Découpage du segment 5 : Frames 561 à 656
Segment 5 enregistré : segments/segment_5.mp4
Découpage terminé. Les segments sont stockés dans le dossier 'segments/'.


#### Analyse des segements

In [8]:
import cv2
import os
import numpy as np

# Fonction pour analyser chaque vidéo
def process_video(video_path, output_path):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Erreur: Impossible de lire la vidéo {video_path}.")
        return

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # Largeur de la vidéo
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # Hauteur de la vidéo
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    print(f"Dimensions de la vidéo {video_path} : {width}x{height}, FPS : {fps}")

    # Initialisation du VideoWriter pour sauvegarder la vidéo annotée
    fourcc = cv2.VideoWriter_fourcc(*'H264')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    prev_frame = None

    while True:
        ret, frame = cap.read()
        if not ret:
            print(f"Fin de la vidéo {video_path} ou erreur de lecture.")
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if prev_frame is not None:
            diff = cv2.absdiff(prev_frame, gray_frame)
            _, thresh = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)

            # Trouver les contours des zones où il y a des différences
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            annotated_frame = frame.copy()
            highest_box = None
            min_y = float('inf')

            # Parcourir les contours et déterminer la boîte avec le plus petit 'y'
            for contour in contours:
                x, y, w, h = cv2.boundingRect(contour)
                if y < min_y:
                    min_y = y
                    highest_box = (x, y, w, h)

            # Si une boîte a été trouvée, dessiner la boîte la plus haute
            if highest_box is not None:
                x, y, w, h = highest_box
                cv2.rectangle(annotated_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(annotated_frame, f"x:{x}, y:{y}", (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                print(f"Coordonnées de la boîte la plus haute : x={x}, y={y}, w={w}, h={h}")

            out.write(annotated_frame)

        prev_frame = gray_frame
        frame_count += 1

    cap.release()
    out.release()

    print(f"Vidéo annotée sauvegardée avec succès : {output_path}")

# Analyser toutes les vidéos dans le dossier 'segments/'
segment_folder = 'segments/'
segment_files = [f for f in os.listdir(segment_folder) if f.endswith('.mp4')]

# Créer un dossier pour les vidéos annotées
os.makedirs("segments_annotated", exist_ok=True)

# Traiter chaque vidéo dans le dossier 'segments/'
for segment_file in segment_files:
    video_path = os.path.join(segment_folder, segment_file)
    output_path = os.path.join("segments_annotated", f"annotated_{segment_file}")
    
    # Analyser et annoter la vidéo
    process_video(video_path, output_path)

cv2.destroyAllWindows()
print("Traitement terminé pour toutes les vidéos.")


Dimensions de la vidéo segments/segment_1.mp4 : 1920x1080, FPS : 30
Coordonnées de la boîte la plus haute : x=685, y=452, w=11, h=11
Coordonnées de la boîte la plus haute : x=686, y=454, w=11, h=11
Coordonnées de la boîte la plus haute : x=1187, y=262, w=1, h=2
Coordonnées de la boîte la plus haute : x=691, y=459, w=8, h=14
Coordonnées de la boîte la plus haute : x=693, y=462, w=7, h=19
Coordonnées de la boîte la plus haute : x=696, y=465, w=3, h=3
Coordonnées de la boîte la plus haute : x=933, y=361, w=1, h=3
Coordonnées de la boîte la plus haute : x=1208, y=372, w=1, h=11
Coordonnées de la boîte la plus haute : x=1208, y=348, w=1, h=3
Coordonnées de la boîte la plus haute : x=691, y=469, w=8, h=8
Coordonnées de la boîte la plus haute : x=1767, y=357, w=1, h=3
Coordonnées de la boîte la plus haute : x=724, y=452, w=10, h=12
Coordonnées de la boîte la plus haute : x=1255, y=207, w=2, h=6
Coordonnées de la boîte la plus haute : x=745, y=418, w=11, h=10
Coordonnées de la boîte la plus ha

### Accelere la video

In [2]:
import cv2

def accelerate_video(input_path, output_path, factor=3):
    # Ouvrir la vidéo d'entrée
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Erreur : Impossible de lire la vidéo.")
        return

    # Obtenir les propriétés de la vidéo
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Nouveau FPS (multiplié par le facteur d'accélération)
    new_fps = fps * factor

    # Configurer l'écrivain de vidéo pour la sortie avec le nouveau FPS
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Format de sortie
    out = cv2.VideoWriter(output_path, fourcc, new_fps, (width, height))

    # Lire et écrire chaque frame à une vitesse plus rapide
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)

    # Libérer les ressources
    cap.release()
    out.release()
    print(f"Vidéo accélérée sauvegardée sous : {output_path}")

# Utilisation du code
input_video = "4.mp4"  # Vidéo d'entrée
output_video = "444.mp4"  # Vidéo accélérée de sortie
accelerate_video(input_video, output_video, factor=3)


Vidéo accélérée sauvegardée sous : 444.mp4


### Clic quand les penealytes sont tirés

In [3]:
import cv2

def record_clicks(video_path, output_file):
    # Ouvrir la vidéo
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Erreur : Impossible de lire la vidéo.")
        return

    clicks = []
    def click_event(event, x, y, flags, param):
        # Si l'utilisateur clique sur la vidéo
        if event == cv2.EVENT_LBUTTONDOWN:
            time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000  # Temps en secondes
            frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))  # Numéro de la frame
            clicks.append(f"{time:.2f} seconds, frame {frame}")
            print(f"Clic enregistré à {time:.2f} secondes, frame {frame}")

    # Afficher la vidéo et attendre les clics
    cv2.namedWindow("Accéléré")
    cv2.setMouseCallback("Accéléré", click_event)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        cv2.imshow("Accéléré", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):  # Appuyer sur 'q' pour quitter
            break

    # Sauvegarder les moments des clics dans un fichier texte
    with open(output_file, 'w') as f:
        for click in clicks:
            f.write(click + "\n")

    # Libérer les ressources
    cap.release()
    cv2.destroyAllWindows()
    print(f"Moments des clics sauvegardés dans : {output_file}")

# Utilisation du code
video_path = "444.mp4"  # Vidéo accélérée
output_file = "4click_times.txt"  # Fichier pour sauvegarder les clics
record_clicks(video_path, output_file)


Clic enregistré à 13.44 secondes, frame 1211
Clic enregistré à 37.98 secondes, frame 3419
Moments des clics sauvegardés dans : 4click_times.txt


### Reforme la video avec seulement les videos 

In [4]:
import cv2

def extract_segments(input_video, click_file, output_video, padding=1):
    # Ouvrir la vidéo d'entrée
    cap = cv2.VideoCapture(input_video)
    if not cap.isOpened():
        print("Erreur : Impossible de lire la vidéo.")
        return

    # Obtenir les propriétés de la vidéo d'entrée
    fps_input = int(cap.get(cv2.CAP_PROP_FPS))  # FPS d'origine
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    
    # Ouvrir le fichier de sortie avec le FPS original
    out = cv2.VideoWriter(output_video, fourcc, fps_input, (width, height))

    # Lire le fichier de clics
    with open(click_file, 'r') as f:
        lines = f.readlines()

    segments = []
    for line in lines:
        time, frame = line.strip().split(', frame ')
        frame_number = int(frame)
        
        # Calcul des frames avant et après le clic, en utilisant la vidéo d'origine
        start_frame = max(0, frame_number - padding * fps_input)  # Prendre 2 sec avant
        end_frame = min(frame_count, frame_number + padding * fps_input)  # Prendre 2 sec après
        segments.append((start_frame, end_frame))

    # Extraire et écrire chaque segment dans la vidéo de sortie
    for start_frame, end_frame in segments:
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        for frame_num in range(start_frame, end_frame):
            ret, frame = cap.read()
            if not ret:
                break
            out.write(frame)

    # Libérer les ressources
    cap.release()
    out.release()
    print(f"Vidéo avec les segments des clics sauvegardée sous : {output_video}")

# Utilisation du code
input_video = "444.mp4"  # Vidéo d'entrée
click_file = "4click_times.txt"  # Fichier des clics
output_video = "final_video_accelerate.mp4"  # Vidéo finale
extract_segments(input_video, click_file, output_video, padding=1)


Vidéo avec les segments des clics sauvegardée sous : final_video_accelerate.mp4


### Deccelere la video 

In [5]:
cap = cv2.VideoCapture('final_video_accelerate.mp4')

fps = cap.get(cv2.CAP_PROP_FPS) 
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  

out = cv2.VideoWriter('final_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps / 3, (width, height))

frame_index = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    out.write(frame)
    frame_index += 1
    if frame_index % 3 != 0: 
        continue

cap.release()
out.release()
cv2.destroyAllWindows()
